In [2]:
import pandas as pd
import re
import numpy as np

In [3]:
dh = pd.read_excel('./DH(임베딩컬럼만).xlsx',index_col=0)
dh

,predict_date,dh_embedding,oil_embedding,target
0,2022-01-02,"[-0.14264455437660217, 0.05831192433834076, 0....","[-0.3298284411430359, -0.40550950169563293, 0....",0
1,2022-01-03,"[-0.15551824867725372, -0.03494337573647499, 0...","[-0.14796359837055206, -0.4272714853286743, 0....",0
2,2022-01-04,"[-0.6354258060455322, -0.365503191947937, 0.98...","[-0.16407831013202667, 0.3745501935482025, 0.2...",0
3,2022-01-05,"[1.2611210346221924, -1.2480064630508423, 0.56...","[0.57369065284729, 0.3639283776283264, 0.44556...",0
4,2022-01-06,"[-1.170987844467163, -0.04770953953266144, 0.0...","[-0.3165379762649536, 0.6350396871566772, 0.40...",0
...,...,...,...,...
479,2023-05-26,"[-0.8856520056724548, 0.06582590192556381, 0.2...","[0.1317824423313141, -0.06475307792425156, 1.1...",0
480,2023-05-28,"[-0.7657447457313538, 1.1281311511993408, -2.1...","[-0.5577801465988159, 0.1386461704969406, 0.52...",1
481,2023-05-29,"[-0.12134005129337311, -0.26261863112449646, 1...","[-0.558170735836029, 0.14962132275104523, 0.47...",1
482,2023-05-30,"[0.152285635471344, -0.14103375375270844, 0.35...","[-0.9299746751785278, 0.0032070425804704428, 0...",1


In [27]:
dh.info()

<class 'pandas.core.frame.DataFrame'>
Index: 484 entries, 0 to 483
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   predict_date   484 non-null    int64  
 1   dh_embedding   484 non-null    float64
 2   oil_embedding  484 non-null    float64
 3   target         484 non-null    int64  
dtypes: float64(2), int64(2)
memory usage: 18.9 KB


In [4]:
# 데이터프레임의 'predict_date' 컬럼을 datetime 타입으로 변환
dh['predict_date'] = pd.to_datetime(dh['predict_date'])

# 'dh_embedding' 컬럼의 값을 문자열에서 리스트로 변환한 후, 각 요소를 float로 변환
dh['dh_embedding'] = dh['dh_embedding'].apply(lambda x: [float(value) for value in x.strip('[]').split(',')])

# 'oil_embedding' 컬럼의 값을 문자열에서 리스트로 변환한 후, 각 요소를 float로 변환
dh['oil_embedding'] = dh['oil_embedding'].apply(lambda x: [float(value) for value in x.strip('[]').split(',')])

# 데이터프레임의 'predict_date', 'dh_embedding', 'oil_embedding' 컬럼의 데이터 타입을 int64로 변경
dh['predict_date'] = dh['predict_date'].astype('int64')
# dh['dh_embedding'] = dh['dh_embedding'].astype('int64')
# dh['oil_embedding'] = dh['oil_embedding'].astype('int64')

In [5]:
dh['dh_embedding'] = np.array([np.mean(embedding) for embedding in dh['dh_embedding']])
dh['oil_embedding'] = np.array([np.mean(embedding) for embedding in dh['oil_embedding']])

In [6]:
import numpy as np
print("데이터의 차원:", dh.ndim)

데이터의 차원: 2


In [7]:
# 모델 라이브러리 로드
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, f1_score, roc_auc_score

In [8]:
from sklearn.model_selection import train_test_split

# 특성과 레이블 분리
X = dh[['dh_embedding', 'oil_embedding']]
y = dh['target']

# 데이터를 훈련 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [20]:
# 모델 함수
def model_basic(X_train, y_train, X_test, y_test):
    models = [
        DecisionTreeClassifier(),
        RandomForestClassifier(),
        XGBClassifier(),
        LGBMClassifier()
    ]

    rdict={'model':[],'accuracy':[],'precision':[],'recall':[],'f1_score':[]}

    
    for clf in models:
        clf = clf.fit(X_train, y_train)
        pred = clf.predict(X_test)
        results = (round(accuracy_score(y_test, pred),2),
                    round(precision_score(y_test, pred),2),
                    round(recall_score(y_test, pred),2),
                    round(f1_score(y_test, pred),2))
        rdict['model'].append(clf); 
        rdict['accuracy'].append(results[0])
        rdict['precision'].append(results[1])
        rdict['recall'].append(results[2])
        rdict['f1_score'].append(results[3])

        # print(results)

    rdf = pd.DataFrame(data=rdict)
    return rdf

In [26]:
X_train

,dh_embedding,oil_embedding
412,0.001257,0.000598
25,0.001411,0.000972
220,0.001636,0.001324
396,0.001740,0.001134
231,0.000960,0.000876
...,...,...
106,0.001260,0.000872
270,0.001245,0.000695
348,0.001610,0.000773
435,0.000742,0.000736


In [22]:
# 모델 함수 호출하기
model_basic(X_train, y_train, X_test, y_test)

,model,accuracy,precision,recall,f1_score
0,DecisionTreeClassifier(),0.49,0.43,0.42,0.42
1,"(DecisionTreeClassifier(max_features='sqrt', r...",0.52,0.44,0.37,0.41
2,"XGBClassifier(base_score=None, booster=None, c...",0.58,0.53,0.44,0.48
3,LGBMClassifier(),0.56,0.50,0.42,0.46


In [23]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# StrandardScaler() Scaler객체 생성.
scaler = StandardScaler()
# 학습 데이터에 대해서 fit(), transform() 수행.
scaler.fit(X_train)
train_scaled_SS = scaler.transform(X_train)
# transform()시 scale 변환된 데이터 셋이 numpy ndarry로 반환되어 이를 DataFrame으로 변환
X_train_SS = pd.DataFrame(data = train_scaled_SS, columns=X_train.columns)

# 테스트 데이터에서는 다시 fit(), transform()이나 fit_transform()을 수행하지 않고 transform만 수행.
test_scaled_SS = scaler.transform(X_test)
# transform()시 scale 변환된 데이터 셋이 numpy ndarry로 반환되어 이를 DataFrame으로 변환
X_test_SS = pd.DataFrame(data = test_scaled_SS, columns=X_test.columns)


# ------------------------------------------ #


# MinMaxScaler() Scaler객체 생성.
scaler = MinMaxScaler()
# 학습 데이터에 대해서 fit(), transform() 수행.
scaler.fit(X_train)
train_scaled_MM = scaler.transform(X_train)
# transform()시 scale 변환된 데이터 셋이 numpy ndarry로 반환되어 이를 DataFrame으로 변환
X_train_MM = pd.DataFrame(data = train_scaled_MM, columns=X_train.columns)

# 테스트 데이터에서는 다시 fit(), transform()이나 fit_transform()을 수행하지 않고 transform만 수행.
test_scaled_MM = scaler.transform(X_test)
# transform()시 scale 변환된 데이터 셋이 numpy ndarry로 반환되어 이를 DataFrame으로 변환
X_test_MM = pd.DataFrame(data = test_scaled_MM, columns=X_test.columns)

In [24]:
model_basic(X_train_SS, y_train, X_test_SS, y_test)

,model,accuracy,precision,recall,f1_score
0,DecisionTreeClassifier(),0.46,0.40,0.44,0.42
1,"(DecisionTreeClassifier(max_features='sqrt', r...",0.48,0.39,0.30,0.34
2,"XGBClassifier(base_score=None, booster=None, c...",0.58,0.53,0.44,0.48
3,LGBMClassifier(),0.56,0.50,0.42,0.46


In [25]:
model_basic(X_train_MM, y_train, X_test_MM, y_test)

,model,accuracy,precision,recall,f1_score
0,DecisionTreeClassifier(),0.44,0.36,0.33,0.34
1,"(DecisionTreeClassifier(max_features='sqrt', r...",0.52,0.45,0.40,0.42
2,"XGBClassifier(base_score=None, booster=None, c...",0.58,0.53,0.44,0.48
3,LGBMClassifier(),0.54,0.47,0.37,0.42


In [15]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(criterion = 'entropy', 
                            max_depth = 10, 
                            min_samples_split = 5, 
                            min_samples_leaf = 10)

# fit
dt.fit(X_train, y_train)

# 예측
y_pred = dt.predict(X_test)


# 모델 성능평가
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(y_test, y_pred)
print(cf_matrix)


# score print
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
print("Accuracy : %.3f" % accuracy_score(y_test, y_pred))
print("Precision : %.3f" % precision_score(y_test, y_pred))
print("Recall : %.3f" % recall_score(y_test, y_pred))
print("F1 : %.3f" % f1_score(y_test, y_pred))

[[32 22]
 [26 17]]
Accuracy : 0.505
Precision : 0.436
Recall : 0.395
F1 : 0.415


In [16]:
from sklearn.svm import SVC
svc = SVC(random_state=0, 
          C = 10, 
          kernel = 'linear')
# C = 클수록 정확하게 (마진이 작아짐, 과대적합) / 작을수록 과대적합 방지
# gamma 클수록 정확하게 과대적합, 작을수록 과대적합 방지 (비선형 rbf, poly 에서만 옵션 사용)

# fit
svc.fit(X_train, y_train)

# 예측
y_pred = svc.predict(X_test)


# 모델 성능평가
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(y_test, y_pred)
print(cf_matrix)


# score print
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
print("Accuracy : %.3f" % accuracy_score(y_test, y_pred))
print("Precision : %.3f" % precision_score(y_test, y_pred))
print("Recall : %.3f" % recall_score(y_test, y_pred))
print("F1 : %.3f" % f1_score(y_test, y_pred))

[[54  0]
 [43  0]]
Accuracy : 0.557
Precision : 0.000
Recall : 0.000
F1 : 0.000


c:\Users\duja1\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators= 500,
                                    criterion= 'entropy',
                                    max_depth= 10,
                                    min_samples_leaf= 5,
                                    max_features= 5,
                                    min_samples_split= 5)

# fit
rf_model.fit(X_train, y_train)

# 예측
y_pred = rf_model.predict(X_test)


# 모델 성능평가
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(y_test, y_pred)
print(cf_matrix)


# score print
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
print("Accuracy : %.3f" % accuracy_score(y_test, y_pred))
print("Precision : %.3f" % precision_score(y_test, y_pred))
print("Recall : %.3f" % recall_score(y_test, y_pred))
print("F1 : %.3f" % f1_score(y_test, y_pred))

[[38 16]
 [32 11]]
Accuracy : 0.505
Precision : 0.407
Recall : 0.256
F1 : 0.314


In [18]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(n_estimators = 150,
                            learning_rate = 0.2,
                            max_depth = 10,
                            min_child_weight = 5,
                            gamma = 10)

# fit
xgb_model.fit(X_train, y_train)

# 예측
y_pred = xgb_model.predict(X_test)


# 모델 성능평가
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(y_test, y_pred)
print(cf_matrix)


# score print
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
print("Accuracy : %.3f" % accuracy_score(y_test, y_pred))
print("Precision : %.3f" % precision_score(y_test, y_pred))
print("Recall : %.3f" % recall_score(y_test, y_pred))
print("F1 : %.3f" % f1_score(y_test, y_pred))

[[54  0]
 [43  0]]
Accuracy : 0.557
Precision : 0.000
Recall : 0.000
F1 : 0.000


c:\Users\duja1\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
from lightgbm import LGBMClassifier

lgbm_model = LGBMClassifier(max_depth=12,
                            learning_rate=0.2,
                            n_estimators=500)

# fit
lgbm_model.fit(X_train, y_train)

# 예측
y_pred = lgbm_model.predict(X_test)


# 모델 성능평가
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(y_test, y_pred)
print(cf_matrix)


# score print
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
print("Accuracy : %.3f" % accuracy_score(y_test, y_pred))
print("Precision : %.3f" % precision_score(y_test, y_pred))
print("Recall : %.3f" % recall_score(y_test, y_pred))
print("F1 : %.3f" % f1_score(y_test, y_pred))

[[33 21]
 [27 16]]
Accuracy : 0.505
Precision : 0.432
Recall : 0.372
F1 : 0.400
